In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pickle
from collections import Counter
from tqdm import tqdm
import itertools
import pandas as pd
from itertools import islice
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import random
!pip install transformers
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Setting device on GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

torch.backends.cudnn.deterministic = True

# Set the random seed manually for reproducibility.
torch.manual_seed(1234)

path = "drive/My Drive/FiQA/"

Using TensorFlow backend.


     |████████████████████████████████| 501kB 3.4MB/s 
     |████████████████████████████████| 870kB 45.5MB/s 
     |████████████████████████████████| 3.7MB 42.2MB/s 
     |████████████████████████████████| 1.0MB 37.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=3d1be901656562faeba2f37e7ada3585b0235dc4fff0bb6b17ea1e191e7c3c36
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [0]:
from evaluate import *

In [0]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def remove_empty(test_set):
    for index, row in enumerate(test_set):
        for doc in row[1]:
            if doc in empty_docs:
                del test_set[index]
    return test_set

def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

def save_pickle(path, data):
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pad_seq(seq, max_seq_len):
    # Pad each seq to be the same length to process in batch.
    # pad_token = 0
    if len(seq) >= max_seq_len:
        seq = seq[:max_seq_len]
    else:
        seq += [0]*(max_seq_len - len(seq))
    return seq

In [0]:
# dict mapping of token to idx
vocab = load_pickle(path + 'vocab_full.pickle')
# dict mapping of docid to doc text
docid_to_text = load_pickle(path + 'label_ans.pickle')

# dict mapping of qid to question text
qid_to_text = load_pickle(path + 'qid_text.pickle')

train_qid_rel = load_pickle(path + "qid_rel_train.pickle")
test_qid_rel = load_pickle(path + "qid_rel_test.pickle")
valid_qid_rel = load_pickle(path + "qid_rel_valid.pickle")

train_set = load_pickle(path + 'data/data_train_50.pickle')
valid_set = load_pickle(path + 'data/data_valid_50.pickle')

test_set = load_pickle(path + 'data/data_test_500_rel.pickle')
test_set_full = load_pickle(path + 'data/data_test_500.pickle')

empty_docs = load_pickle(path+'empty_docs.pickle')

In [0]:
train_set = [x for x in train_set if x[1] not in empty_docs]
valid_set = [x for x in valid_set if x[1] not in empty_docs]

test_set = remove_empty(test_set)
test_set_full = remove_empty(test_set_full)

print("Number of training samples: {}".format(len(train_set)))
print("Number of validation samples: {}".format(len(valid_set)))
print("Number of test samples: {}".format(len(test_set)))

Number of training samples: 283707
Number of validation samples: 31582
Number of test samples: 330


In [0]:
collection = pd.read_csv(path+"data-bert/collection_new.tsv", sep="\t", header=None)
collection = collection.rename(columns={0: 'docid', 1: 'doc'})

def load_questions(path):
    """
    Returns a dataframe of cols: qid, question
    """
    # Question ID and Question text
    query_df = pd.read_csv(path, sep="\t")
    queries = query_df[['qid', 'question']]

    return queries

queries = load_questions(path + "FiQA_train_question_final.tsv")

# Question to question text
qid_to_text = {}

for index, row in queries.iterrows():
    qid_to_text[row['qid']] = row['question']

docid_to_text = {}

for index, row in collection.iterrows():
    docid_to_text[row['docid']] = row['doc']

In [0]:
# # Load the BERT tokenizer.
# print('Loading BERT tokenizer...')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [0]:
from tokenizers import BertWordPieceTokenizer
from tokenizers.processors import BertProcessing

tokenizer = BertWordPieceTokenizer(path+"model/fin_model/vocab.txt")
tokenizer._tokenizer.post_processor = BertProcessing(
    ("[SEP]", tokenizer.token_to_id("[CLS]")),
    ("[CLS]", tokenizer.token_to_id("[SEP]")),
)
tokenizer.enable_truncation(max_length=512)
tokenizer.enable_padding(max_length=512)

In [0]:
# label_to_ans = load_pickle(path+"data-bert/label_to_ans.pickle")
# qid_to_text = load_pickle(path+"data-bert/qid_to_text.pickle")

In [0]:
def add_ques_token(string):
    question = string + " [SEP] "

    return question

In [0]:
def get_sequence_df(dataset):
    df = pd.DataFrame(dataset)
    df = df.rename(columns={0: 'qid', 1: 'pos', 2:'neg'})
    df_pos = df[['qid', 'pos']]
    df_pos = df_pos.rename(columns={'pos': 'docid'})
    df_pos['label'] = df_pos.apply(lambda x: 1, axis=1)
    df_pos = df_pos.drop_duplicates()

    df_neg = df[['qid', 'neg']]
    df_neg = df_neg.rename(columns={'neg': 'docid'})
    df_neg['label'] = df_neg.apply(lambda x: 0, axis=1)
    data_df = pd.concat([df_pos, df_neg]).sort_values(by=['qid'])

    data_df['question'] = data_df['qid'].apply(lambda x: qid_to_text[x])
    data_df['ans_cand'] = data_df['docid'].apply(lambda x: docid_to_text[x])

    data_df['question'] = data_df['question'].apply(add_ques_token)
    data_df['seq'] = data_df['question'] + data_df['ans_cand']
    # dataset = dataset[['seq']]
    # data_df['ques_token'] = data_df['question'].apply(lambda x: add_question_token(x))
    # data_df['ans_cand'] = data_df['ans_cand'].apply(lambda x: add_ans_token(x))

    # data_df = data_df[['qid', 'docid', 'label', 'ans_cand','ques_token']]
    # data_df['seq'] = data_df['ques_token'] + data_df['ans_cand']

    # data_df['seq_clipped'] = data_df['seq'].apply(clip)
    # # train['len'] = train['seq_clipped'].apply(lambda x: len(x))

    return data_df

In [0]:
# Get the lists of sentences and their labels.
train_data = get_sequence_df(train_set)
train_sequences = train_data.seq.values
train_labels = train_data.label.values

# a = train_sequences[0]

# tokenized_seq = tokenizer.encode(a)

train_inputs = []
train_att_masks = []


for i, seq in enumerate(train_sequences):

    tokenized_seq = tokenizer.encode(seq)

    train_inputs.append(tokenized_seq.ids)
    train_att_masks.append(tokenized_seq.attention_mask)

In [0]:
path

'drive/My Drive/FiQA/'

In [0]:
save_pickle(path+"data-finbert/train_inputs.pickle", train_inputs)
save_pickle(path+"data-finbert/train_att_masks.pickle", train_att_masks)
save_pickle(path+"data-finbert/train_labels.pickle", train_labels)


In [0]:
train_inputs = load_pickle(path+"data-finbert/train_inputs.pickle")
train_att_masks = load_pickle(path+"data-finbert/train_att_masks.pickle")
train_labels = load_pickle(path+"data-finbert/train_labels.pickle")


In [0]:
# Get the lists of sentences and their labels.
valid_data = get_sequence_df(valid_set)
valid_sequences = valid_data.seq.values
valid_labels = valid_data.label.values

# a = train_sequences[0]

# tokenized_seq = tokenizer.encode(a)

valid_inputs = []
valid_att_masks = []


for i, seq in enumerate(valid_sequences):

    tokenized_seq = tokenizer.encode(seq)

    valid_inputs.append(tokenized_seq.ids)
    valid_att_masks.append(tokenized_seq.attention_mask)

In [0]:
print(len(train_inputs))
# print(len(valid_inputs))

298401


In [0]:
# train_inputs = train_inputs[:10000]
# train_att_masks = train_att_masks[:10000]
# train_labels = train_labels[:10000]

# valid_inputs = train_inputs[:100]
# valid_att_masks = train_att_masks[:100]
# valid_labels = train_labels[:100]

## **Pointwise**

In [0]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_input = torch.tensor(train_inputs)
# validation_input = torch.tensor(valid_inputs)

train_label = torch.tensor(train_labels)
# validation_label = torch.tensor(valid_labels)

train_mask = torch.tensor(train_att_masks)
# validation_mask = torch.tensor(valid_att_masks)

In [0]:
print(len(train_input))
# print(len(validation_input))

298401


In [0]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of
# 16 or 32.

batch_size = 8

# Create the DataLoader for our training set.
training_data = TensorDataset(train_input, train_mask, train_label)
train_sampler = RandomSampler(training_data)
train_dataloader = DataLoader(training_data, sampler=train_sampler, batch_size=batch_size)

# # Create the DataLoader for our validation set.
# validation_data = TensorDataset(validation_input, validation_mask, validation_label)
# validation_sampler = SequentialSampler(validation_data)
# validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
print(len(train_dataloader))
# print(len(validation_dataloader))

37301


In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## **Model**

In [0]:
import torch.nn as nn

class BertClassifier(nn.Module):
    def __init__(self, bert):
        
        super().__init__()

        self.config = BertConfig.from_pretrained(path+"model/fin_model/config.json")
        # self.config = BertConfig.from_pretrained("/content/drive/My Drive/FiQA/bert-lm/weights2/config.json")
        # self.config = BertConfig()
        self.num_labels = self.config.num_labels
        self.bert = bert
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.config.hidden_size, self.config.num_labels)

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):

        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
# model = BertForSequenceClassification.from_pretrained(path+"model/fin_model")
# model.to(device)

In [0]:
# bert = BertModel.from_pretrained('bert-base-uncased')

# # model_path = "/content/drive/My Drive/FiQA/bert-lm/test_lm/"
# # bert = BertModel.from_pretrained(model_path)

# model = BertClassifier(bert)

# # Tell pytorch to run this model on the GPU.
# model.to(device)

In [0]:
# bert = BertModel.from_pretrained('bert-base-uncased')

# model_path = "/content/drive/My Drive/FiQA/bert-lm/weights2/"
model_path = path+"model/fin_model/"
bert = BertModel.from_pretrained(model_path)

model = BertClassifier(bert)

# Tell pytorch to run this model on the GPU.
model.to(device)

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [0]:
def train(model, train_dataloader, optimizer):

    # Store the average loss after each epoch so we can plot them.
    loss_values = []

    # Reset the total loss for this epoch.
    total_loss = 0
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(tqdm(train_dataloader)):

        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        logits = outputs[1]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

        # Accumulate the training loss over all of the batches
        total_loss += loss.item()
    
        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        optimizer.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    acc = eval_accuracy/nb_eval_steps

    return avg_train_loss, acc

In [0]:
def validate(model, validation_dataloader):

    model.eval()

    # Tracking variables 
    total_loss = 0
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in tqdm(validation_dataloader):
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,
                            labels=b_labels)
        
        loss = outputs[0]

        logits = outputs[1]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

        total_loss += loss.item()

    acc = eval_accuracy/nb_eval_steps
    avg_loss = total_loss / len(validation_dataloader) 

    return avg_loss, acc

In [0]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [0]:
# Lowest validation lost
best_valid_loss = float('inf')

n_epochs = 1

for epoch in range(n_epochs):

    # Evaluate training loss
    train_loss, train_acc = train(model, train_dataloader, optimizer)
    
    # Evaluate validation loss
    # valid_loss, valid_acc = validate(model, validation_dataloader)
    
    # # At each epoch, if the validation loss is the best
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    # torch.save(model.state_dict(), path + 'model/' + str(epoch+1)+'_finbert-full.pt')
    torch.save(model.state_dict(), path + 'model/1_finbert-full.pt')

    print("\n\n Epoch {}:".format(epoch+1))
    print("\t Train Loss: {} | Train Accuracy: {}%".format(round(train_loss, 3), round(train_acc*100, 2)))
    # print("\t Validation Loss: {} | Validation Accuracy: {}%\n".format(round(valid_loss, 3), round(valid_acc*100, 2)))

 21%|██        | 7656/37301 [57:25<3:42:45,  2.22it/s]

In [0]:
torch.save(model.state_dict(), path + 'model/2_model-bert-512.pt')

In [0]:
print(torch.cuda.get_device_name(0))
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.4 GB
Cached:    0.5 GB


In [0]:
test_set = load_pickle(path + 'data/data_test_500_rel.pickle')
test_set_full = load_pickle(path + 'data/data_test_500.pickle')

empty_docs = load_pickle(path+'empty_docs.pickle')

test_set = remove_empty(test_set)
test_set_full = remove_empty(test_set_full)


In [0]:
for row in test_set:
    row[2] = [x for x in row[2] if x is not 0]

for row in test_set_full:
    row[2] = [x for x in row[2] if x is not 0]

In [0]:
# test_set2 = test_set

# for row in test_set2:
#     row[2] = random.sample(row[2], len(row[2]))

In [0]:
count = 0

for row in test_set_full:
    if not any(x in row[2]  for x in row[1]):
        count += 1

print(count)

49


In [0]:
count = 0

for row in test_set_full:
    if not all(x in row[2]  for x in row[1]):
        count += 1

print(count)

162


In [0]:
len(test_set_full)

330

In [0]:
def get_sequence_df(dataset):
    df = pd.DataFrame(dataset)
    df = df.rename(columns={0: 'qid', 1: 'pos', 2:'neg'})
    df_pos = df[['qid', 'pos']]
    df_pos = df_pos.rename(columns={'pos': 'docid'})
    df_pos['label'] = df_pos.apply(lambda x: 1, axis=1)
    df_pos = df_pos.drop_duplicates()

    df_neg = df[['qid', 'neg']]
    df_neg = df_neg.rename(columns={'neg': 'docid'})
    df_neg['label'] = df_neg.apply(lambda x: 0, axis=1)
    data_df = pd.concat([df_pos, df_neg]).sort_values(by=['qid'])

    data_df['question'] = data_df['qid'].apply(lambda x: qid_to_text[x])
    data_df['ans_cand'] = data_df['docid'].apply(lambda x: docid_to_text[x])

    data_df['question'] = data_df['question'].apply(add_ques_token)
    data_df['seq'] = data_df['question'] + data_df['ans_cand']
    # dataset = dataset[['seq']]
    # data_df['ques_token'] = data_df['question'].apply(lambda x: add_question_token(x))
    # data_df['ans_cand'] = data_df['ans_cand'].apply(lambda x: add_ans_token(x))

    # data_df = data_df[['qid', 'docid', 'label', 'ans_cand','ques_token']]
    # data_df['seq'] = data_df['ques_token'] + data_df['ans_cand']

    # data_df['seq_clipped'] = data_df['seq'].apply(clip)
    # # train['len'] = train['seq_clipped'].apply(lambda x: len(x))

    return data_df

In [0]:
test_df = pd.DataFrame(test_set_full)
test_df = test_df.rename(columns={0: 'qid', 1: 'pos', 2:'cand'})
# test_df = test_df[['qid', 'cand']]

test_pos = test_df[['qid', 'pos']]
test_pos = test_pos.explode('pos')
test_pos = test_pos.rename(columns={'pos': 'docid'})
test_pos['label'] = test_pos.apply(lambda x: 1, axis=1)

test_neg = test_df[['qid', 'cand']]
test_neg = test_neg.explode('cand')
test_neg = test_neg.rename(columns={'cand': 'docid'})
test_neg ['label'] = test_neg .apply(lambda x: 0, axis=1)

test_neg.head(5)

test_data = pd.concat([test_pos, test_neg]).sort_values(by=['qid'])

test_data['question'] = test_data['qid'].apply(lambda x: qid_to_text[x])
test_data['ans_cand'] = test_data['docid'].apply(lambda x: docid_to_text[x])

test_data['ques_token'] = test_data['question'].apply(lambda x: add_ques_token(x))
# test_data['ans_cand'] = test_data['ans_cand'].apply(lambda x: add_ans_token(x))

# test_data = test_data[['qid', 'docid', 'label', 'ans_cand','ques_token']]
test_data['seq'] = test_data['ques_token'] + test_data['ans_cand']
# test_data['seq_clipped'] = test_data['seq'].apply(clip)

# test_data.head(5)

# docid_map = test_data[['docid', 'seq_clipped']]
test_full_lm_docid_to_seq = {}

for index, row in test_data.iterrows():
    test_full_lm_docid_to_seq[row['docid']] = row['seq']

print(take(5, test_full_lm_docid_to_seq.items()))

[(14255, "What is the easiest way to back-test index funds and ETFs? [SEP] Yes you can claim your business deductions if you are not making any income yet. But first you should decide what structure you want to have for your business. Either a Company structure or a Sole Trader or Partnership. Company Structure If you choose a Company Structure (which is more expensive to set up) you would claim your deductions but no income. So you would be making a loss, and continue making losses until your income from the business exceed your expenses. So these losses will remain inside the Company and can be carried forward to future income years when you are making profits to offset these profits. Refer to ATO - Company tax losses for more information. Sole Trader of Partnership Structure If you choose to be a Sole Trader or a Partnership and your business makes a loss you must check the non-commercial loss rules to see if you can offset the loss against your income from other sources, such as wa

In [0]:
save_pickle(path+'data-bert/test_full_lm_docid_to_seq.pickle', test_full_lm_docid_to_seq)

In [0]:
# test_docid_to_seq = load_pickle(path+'data-bert/test_docid_to_seq.pickle')
# test_docid_to_seq = load_pickle(path+'data-bert/test_full_docid_to_seq.pickle')
test_docid_to_seq = load_pickle(path+'data-bert/test_lm_docid_to_seq.pickle')
# test_docid_to_seq = load_pickle(path+'data-bert/test_full_lm_docid_to_seq.pickle')

In [0]:
def get_rank(model, test_set, qid_rel, max_seq_len):

    qid_pred_rank = {}

    model.eval()

    for i, seq in enumerate(tqdm(test_set)):
        
        qid, label, cands = seq[0], seq[1], seq[2]

        cands_id = np.array(cands)

        scores = []

        for docid in cands:

            seq_text = test_docid_to_seq[docid]

            tokenized_seq = tokenizer.encode(seq_text)

            input_ids = tokenized_seq.ids

            att_mask = torch.tensor([tokenized_seq.attention_mask]).to(device)
            
            input_ids = torch.tensor([input_ids]).to(device)

            with torch.no_grad():
            # Forward pass, calculate logit predictions
                outputs = model(input_ids, token_type_ids=None, attention_mask=att_mask)

            logits = outputs[0]

            pred = torch.sigmoid(logits)

            # Move logits and labels to CPU
            pred = pred.detach().cpu().numpy()
            scores.append(pred[:,1][0])

        # print(scores)

        # Get the indices of the sorted similarity scores
        sorted_index = np.argsort(scores)[::-1]

        # Get the docid from the sorted indices
        ranked_ans = cands_id[sorted_index]

        # Dict - key: qid, value: ranked list of docids
        qid_pred_rank[qid] = ranked_ans

    return qid_pred_rank
    # MRR, average_ndcg, precision = evaluate(qid_pred_rank, qid_rel, k)

    # return qid_pred_rank, MRR, average_ndcg, precision

In [0]:
print(test_set2[0])

[1, [14255], [136804, 596289, 363495, 223624, 212661, 481114, 278460, 377547, 352640, 109546, 203905, 42665, 197352, 285255, 179144, 559671, 528564, 8200, 76618, 487551, 72053, 420295, 327002, 176054, 216008, 362069, 33544, 361978, 224167, 311285, 397920, 129355, 108062, 331898, 472824, 69317, 204554, 98072, 15270, 98636, 399115, 18539, 78486, 392484, 40628, 153377, 360925, 202457, 99745, 391619, 5762, 245447, 167494, 453455, 509346, 141738, 314161, 391841, 541809, 359814, 248629, 290265, 283113, 151645, 201546, 521044, 364938, 561750, 540395, 187384, 584232, 294738, 307779, 97489, 452248, 274870, 544381, 542213, 191848, 146657, 589416, 16911, 55500, 328745, 316074, 202645, 177074, 402437, 79411, 420846, 131453, 492456, 245011, 484231, 243822, 46791, 12822, 113876, 245810, 541682, 38249, 30343, 531442, 375357, 174714, 121063, 377621, 132780, 427849, 483664, 422199, 576218, 202179, 34810, 331384, 193367, 237207, 333954, 462184, 276934, 291079, 466718, 87113, 574417, 354314, 418999, 2494

In [0]:
toy_test_label = dict(itertools.islice(test_qid_rel.items(), 100))
toy_test = test_set[:100]

In [0]:
model.load_state_dict(torch.load(path+'model/2_finbert-test.pt'))

<All keys matched successfully>

In [0]:
# model.load_state_dict(torch.load(path+'model/2_model-bert-512.pt'))

# qid_pred_rank = get_rank(model, test_set_full, test_qid_rel, max_seq_len=512)
qid_pred_rank = get_rank(model, toy_test, toy_test_label, max_seq_len=512)
# qid_pred_rank2 = get_rank(model,toy_test2, toy_test_label, max_seq_len=512)

100%|██████████| 100/100 [18:35<00:00, 11.10s/it]


In [0]:
k = 10

num_q = len(test_set)

# MRR, average_ndcg, precision = evaluate(qid_pred_rank, test_qid_rel, k)
MRR, average_ndcg, precision = evaluate(qid_pred_rank, toy_test_label, k)

print("\n\nAverage nDCG@{} for {} queries: {}\n".format(k, num_q, average_ndcg))

print("MRR@{} for {} queries: {}\n".format(k, num_q, MRR))

print("Average Precision@{}: {}".format(1, precision))



Average nDCG@10 for 330 queries: 0.023979694423373953

MRR@10 for 330 queries: 0.010317460317460315

Average Precision@1: 0.0


In [0]:
save_pickle(path+'rank/rank_lm_bert_test_full.pickle', qid_pred_rank)